# H2O Auto ML

In [2]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "19" 2022-09-20; OpenJDK Runtime Environment (build 19+36-2238); OpenJDK 64-Bit Server VM (build 19+36-2238, mixed mode, sharing)
  Starting server from /home/shishiriyer/.local/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnbwp5cum
  JVM stdout: /tmp/tmpnbwp5cum/h2o_shishiriyer_started_from_python.out
  JVM stderr: /tmp/tmpnbwp5cum/h2o_shishiriyer_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_shishiriyer_xxisoh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.770 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
# Load the dataset
train_df = pd.read_csv('archive/train.csv')
test_df = pd.read_csv('archive/test_public.csv')
coord_lookup = pd.read_csv('archive/metaData_taxistandsID_name_GPSlocation.csv')

In [5]:
def get_lon(station_no):
    return float(coord_lookup.loc[coord_lookup['ID'] == station_no]['Longitude'].iloc[0])

def get_lat(station_no):
    return float(coord_lookup.loc[coord_lookup['ID'] == station_no]['Latitude'].iloc[0])

In [30]:
encoder = LabelEncoder()
train_df['CALL_TYPE'] = encoder.fit_transform(train_df['CALL_TYPE'])

# Normalize timestamp
train_df['TIMESTAMP'] = pd.to_datetime(train_df['TIMESTAMP'], unit='s')
train_df['MINUTE'] = train_df['TIMESTAMP'].dt.minute
train_df['HOUR'] = train_df['TIMESTAMP'].dt.hour
train_df['DAY'] = train_df['TIMESTAMP'].dt.dayofweek
train_df['MONTH'] = train_df['TIMESTAMP'].dt.month

# Split the training data based on whether CALL_TYPE == 'B' (i.e. there is an origin stand present)
has_origin = train_df['CALL_TYPE'] == 1
train_df_origin = train_df[has_origin]
train_df_nan = train_df[~has_origin]
train_df_origin = train_df_origin.dropna(subset=['ORIGIN_STAND'])

features_nan = pd.DataFrame({ 
    'MINUTE': train_df_nan['MINUTE'], 
    'HOUR': train_df_nan['HOUR'], 
    'DAY': train_df_nan['DAY'], 
    'MONTH': train_df_nan['MONTH'], 
    'TIME': train_df_nan['POLYLINE'].str.count(',') * 15 
})


features_origin = pd.DataFrame({
    'HOUR': train_df_origin['HOUR'], 
    'DAY': train_df_origin['DAY'], 
    'MONTH': train_df_origin['MONTH'], 
    'STAND': train_df_origin['ORIGIN_STAND'],
    'TIME': train_df_origin['POLYLINE'].str.count(',') * 15
})

print(features_nan)

         MINUTE  HOUR  DAY  MONTH  TIME
0             0     0    0      7   675
2             2     0    0      7  1935
3             0     0    0      7  1275
4             4     0    0      7   855
5             2     0    0      7   765
...         ...   ...  ...    ...   ...
1710663       0    11    3      1  5805
1710664      16    15    2      1     0
1710665      37    23    0      6   945
1710666      36    23    0      6   885
1710667      41    10    4      1     0

[892789 rows x 5 columns]


In [31]:
features_nan = h2o.H2OFrame(features_nan)
features_origin = h2o.H2OFrame(features_origin)
train_nan, test_nan = features_nan.split_frame(ratios=[0.8], seed = 1)
train_origin, test_origin = features_origin.split_frame(ratios=[0.8], seed=1)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [32]:
aml_nan = H2OAutoML(max_models=25, seed=1, nfolds=0)
aml_nan.train(training_frame=train_nan, y='TIME', validation_frame=test_nan)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid_1_AutoML_8_20230605_05742_model_3


Model Summary: 
    number_of_trees
--  -----------------
    35

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 2635618.7888070396
RMSE: 1623.458896556066
MAE: 796.8260421329493
RMSLE: 1.152469193993221
Mean Residual Deviance: 2635618.7888070396

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 2744080.791641871
RMSE: 1656.5267253026348
MAE: 798.8788498821393
RMSLE: 1.1529997255381579
Mean Residual Deviance: 2744080.791641871

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance    validation_rmse    validation_mae    validation_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------  -----------------  ----------------  ---------------------
    2023-06-05 01:00:11  21.825 sec  0                  2253.47          1541.56         5.07812e+06          2273.79            1539.71           5.17012e+06
    2023-06-05 01:00:12  22.975 sec  5                  1648.56          764.721         2.71776e+06          1677.87            765.468           2.81525e+06
    2023-06-05 01:00:14  24.135 sec  10                 1626.99          787.667         2.6471e+06           1657.33            788.558           2.74674e+06
    2023-06-05 01:00:15  25.804 sec  15                 1625.62          795.778         2.64264e+06          1656.66            796.918           2.74453e+06
    2023-06-05 01:00:17  27.347 sec  20                 1624.93          797.038         2.64041e+06          1656.58            798.38            2.74427e+06
    2023-06-05 01:00:18  28.982 sec  25                 1624.46          797.08          2.63887e+06          1656.51            798.646           2.74401e+06
    2023-06-05 01:00:20  30.823 sec  30                 1623.9           796.946         2.63706e+06          1656.5             798.725           2.74398e+06
    2023-06-05 01:00:22  32.902 sec  35                 1623.46          796.826         2.63562e+06          1656.53            798.879           2.74408e+06

Variable Importances: 
variable    relative_importance    scaled_importance    percentage
----------  ---------------------  -------------------  ------------
HOUR        6.54508e+10            1                    0.706931
DAY         1.07819e+10            0.164733             0.116455
MONTH       1.06639e+10            0.16293              0.11518
MINUTE      5.68788e+09            0.0869032            0.0614345

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [28]:
leaderboard = aml_nan.leaderboard
print(leaderboard.head())

model_id                                           rmse          mse      mae     rmsle    mean_residual_deviance
XGBoost_grid_1_AutoML_7_20230605_03457_model_3  1323.54  1.75176e+06  682.064  0.990681               1.75176e+06
GBM_3_AutoML_7_20230605_03457                   1323.72  1.75224e+06  682.091  0.99124                1.75224e+06
GBM_4_AutoML_7_20230605_03457                   1323.82  1.7525e+06   682.251  0.99122                1.7525e+06
GBM_1_AutoML_7_20230605_03457                   1323.95  1.75285e+06  682.545  0.990693               1.75285e+06
GBM_2_AutoML_7_20230605_03457                   1324.01  1.75301e+06  682.408  0.991608               1.75301e+06
XGBoost_3_AutoML_7_20230605_03457               1324.04  1.75309e+06  682.521  0.991391               1.75309e+06
XGBoost_grid_1_AutoML_7_20230605_03457_model_1  1324.08  1.7532e+06   682.744  0.991238               1.7532e+06
XGBoost_grid_1_AutoML_7_20230605_03457_model_5  1324.11  1.75326e+06  682.739  0.991619   

In [33]:
aml_origin = H2OAutoML(max_models=25, seed=1, nfolds=0)
aml_origin.train(training_frame=train_origin, y='TIME', validation_frame=test_origin)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid_1_AutoML_9_20230605_11646_model_3


Model Summary: 
    number_of_trees
--  -----------------
    60

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 876840.641186691
RMSE: 936.3976939242701
MAE: 539.3707271828101
RMSLE: 0.7675937488635143
Mean Residual Deviance: 876840.641186691

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 820046.7472490408
RMSE: 905.5643252961331
MAE: 539.6410701875657
RMSLE: 0.7681325387256137
Mean Residual Deviance: 820046.7472490408

Scoring History: 
    timestamp            duration    number_of_trees    training_rmse    training_mae    training_deviance    validation_rmse    validation_mae    validation_deviance
--  -------------------  ----------  -----------------  ---------------  --------------  -------------------  -----------------  ----------------  ---------------------
    2023-06-05 01:19:42  37.734 sec  0                  1655.1           1342.47         2.73937e+06          1634.87            1341.2            2.67279e+06
    2023-06-05 01:19:42  38.484 sec  5                  975.418          541.138         951440               942.52             539.288           888344
    2023-06-05 01:19:43  39.196 sec  10                 945.138          540.208         893285               911.66             539.242           831125
    2023-06-05 01:19:44  40.061 sec  15                 942.554          543.409         888408               909.159            542.558           826571
    2023-06-05 01:19:45  41.160 sec  20                 941.054          542.901         885582               907.886            542.155           824257
    2023-06-05 01:19:46  42.475 sec  25                 939.961          542.127         883526               907.176            541.493           822969
    2023-06-05 01:19:48  43.887 sec  30                 939.198          541.489         882093               906.713            540.951           822128
    2023-06-05 01:19:49  45.425 sec  35                 938.583          541.055         880938               906.479            540.66            821704
    2023-06-05 01:19:51  47.241 sec  40                 938.029          540.591         879898               906.213            540.349           821222
    2023-06-05 01:19:53  49.212 sec  45                 937.608          540.198         879108               906.008            540.116           820851
    2023-06-05 01:19:55  51.362 sec  50                 937.231          539.88          878402               905.844            539.963           820554
    2023-06-05 01:19:58  53.963 sec  55                 936.804          539.616         877601               905.727            539.794           820342
    2023-06-05 01:20:01  57.167 sec  60                 936.398          539.371         876841               905.564            539.641           820047

Variable Importances: 
variable    relative_importance    scaled_importance    percentage
----------  ---------------------  -------------------  ------------
HOUR        3.50348e+10            1                    0.459141
STAND       2.56331e+10            0.731648             0.33593
DAY         7.85246e+09            0.224133             0.102909
MONTH       7.78463e+09            0.222197             0.10202

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [37]:
test_df['CALL_TYPE'] = encoder.fit_transform(test_df['CALL_TYPE'])

# Normalize timestamp
test_df['TIMESTAMP'] = pd.to_datetime(test_df['TIMESTAMP'], unit='s')
test_df['MINUTE'] = test_df['TIMESTAMP'].dt.minute
test_df['HOUR'] = test_df['TIMESTAMP'].dt.hour
test_df['DAY'] = test_df['TIMESTAMP'].dt.dayofweek
test_df['MONTH'] = test_df['TIMESTAMP'].dt.month

# Split the training data based on whether CALL_TYPE == 'B' (i.e. there is an origin stand present)
has_origin = test_df['CALL_TYPE'] == 1
test_df_origin = test_df[has_origin]
test_df_nan = test_df[~has_origin]
test_df_origin = test_df_origin.dropna(subset=['ORIGIN_STAND'])

features_nan = pd.DataFrame({ 
    'MINUTE': test_df_nan['MINUTE'], 
    'HOUR': test_df_nan['HOUR'], 
    'DAY': test_df_nan['DAY'], 
    'MONTH': test_df_nan['MONTH']
})


features_origin = pd.DataFrame({
    'HOUR': test_df_origin['HOUR'], 
    'DAY': test_df_origin['DAY'], 
    'MONTH': test_df_origin['MONTH'], 
    'STAND': test_df_origin['ORIGIN_STAND']
})

print(features_origin)

     HOUR  DAY  MONTH  STAND
0      17    3      8   15.0
1      17    3      8   57.0
2      17    3      8   15.0
3      17    3      8   53.0
4      17    3      8   18.0
..    ...  ...    ...    ...
290    14    6     12   16.0
295    14    6     12   42.0
297    14    6     12   53.0
310    14    6     12   22.0
316    14    6     12   53.0

[123 rows x 4 columns]


## WIP

In [38]:
features_nan = h2o.H2OFrame(features_nan)
features_origin = h2o.H2OFrame(features_origin)

import csv

fields = ['TRIP_ID', 'TRAVEL_TIME']
rows = []

i_n = 0
i_o = 0

for i in range(320):
    trip_id = test_df.iloc[i, 0]
    row = test_df.iloc[i]
    # print(i_n, i_o)
    if row['CALL_TYPE'] == 1:
        # print(features_origin.iloc[i_o].to_numpy().reshape(1, -1))
        rows.append([trip_id, aml_origin.predict(features_origin.iloc[i_o].to_numpy().reshape(1, -1))[0]])
        i_o += 1
    else:
        # print(features_nan.iloc[i_n].to_numpy().reshape(1, -1))
        rows.append([trip_id, aml_nan.predict(features_nan.iloc[i_n].to_numpy().reshape(1, -1))[0]])
        i_n += 1
    
with open("submission.csv", 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(rows)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


AttributeError: 'H2OFrame' object has no attribute 'iloc'

In [ ]:
h2o.shutdown()